In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
%matplotlib qt
import matplotlib.dates as mdates
from glob import glob

In [2]:
path = '/media/onno/Volume/GFS_T850/'

In [3]:
coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine



Combine all anomalies for all events into one csv dataframe

In [132]:

datez_pw = 'pw_lon_{}_{}_lat_{}_{}.npy'
datez_cw = 'cw_lon_{}_{}_lat_{}_{}.npy'
datez_average = 'avg_p40_p60_lon_{}_{}_lat_{}_{}.npy'
datez_all = [datez_average]
extremez = ['average']
fcst_modelz = ['ERA5RF']
for fcst_model in fcst_modelz:
    for lat_1,lat_0,lon_0,lon_1 in coordinatez:
        for i,extreme in enumerate(extremez):
            datez = datez_all[i].format(lon_0,lon_1,lat_1,lat_0)
            datez = np.load(path+'/dates/'+datez,allow_pickle=True)
            rean_anomz = []
            fcst_anomz = []
            for date in datez[:,0]:
                date = pd.Timestamp(date)
                rean_file = '/anom_files/rean_lon_{}_{}_lat_{}_{}_{}_{:02d}_{:02d}_{}.nc'.format(lon_0,lon_1,lat_1,lat_0,
                                                                            date.year,date.month,date.day,
                                                                            extreme)
                fcst_file = '/anom_files/fcst_lon_{}_{}_lat_{}_{}_{}_{:02d}_{:02d}_{}.nc'.format(lon_0,lon_1,lat_1,lat_0,
                                                                            date.year,date.month,date.day,
                                                                            extreme)
                try:
                    rean_sub = xr.open_dataset(path+rean_file)
                    fcst_sub = xr.open_dataset(path+fcst_file)
                    rean_anomz.append(list(rean_sub.t.values))
                    fcst_anomz.append(list(fcst_sub.t.values))
                except FileNotFoundError:
                    rean_anomz.append(list(np.full(10,np.nan)))
                    fcst_anomz.append(list(np.full(10,np.nan)))                    

            df_rean = pd.DataFrame(data=rean_anomz,index=datez[:,0],columns=np.arange(1,11))
            df_fcst = pd.DataFrame(data=fcst_anomz,index=datez[:,0],columns=np.arange(1,11))
            df_rean.to_csv(path+'/anom_files/rean_lon_{}_{}_lat_{}_{}_anoms_{}.csv'.format(lon_0,lon_1,lat_1,lat_0,extreme))
            df_fcst.to_csv(path+'/anom_files/fcst_lon_{}_{}_lat_{}_{}_anoms_{}.csv'.format(lon_0,lon_1,lat_1,lat_0,extreme))


Plot persistent warm extremes

In [8]:
for lat_1,lat_0,lon_0,lon_1 in coordinatez:
    fig,ax = plt.subplots(figsize=(12,9))
    df_rean = pd.read_csv(path+'/anom_files/rean_lon_{}_{}_lat_{}_{}_anoms_pw.csv'.format(lon_0,lon_1,lat_1,lat_0),index_col=0)
    df_fcst = pd.read_csv(path+'/anom_files/fcst_lon_{}_{}_lat_{}_{}_anoms_pw.csv'.format(lon_0,lon_1,lat_1,lat_0),index_col=0)
    df_rean_median = df_rean.median(axis=0)
    df_fcst_median = df_fcst.median(axis=0)
    df_rean_q = df_rean.quantile([0.1,0.25,0.75,0.9])
    df_fcst_q = df_fcst.quantile([0.1,0.25,0.75,0.9])
    ax.plot(np.arange(1,11),df_rean_median,label='Reanalysis')
    ax.plot(np.arange(1,11),df_fcst_median,label='Forecast')

    ax.fill_between(np.arange(1,11),df_rean_q.loc[0.75],df_rean_median,color='#1f77b4',alpha=.3)
    ax.fill_between(np.arange(1,11),df_rean_q.loc[0.25],df_rean_median,color='#1f77b4',alpha=.3)
    ax.plot(np.arange(1,11),df_rean_q.loc[0.9],color='#1f77b4',alpha=.5,linestyle='--')
    ax.plot(np.arange(1,11),df_rean_q.loc[0.1],color='#1f77b4',alpha=.5,linestyle='--')
    
    ax.fill_between(np.arange(1,11),df_fcst_q.loc[0.75],df_fcst_median,color='#ff7f0e',alpha=.3)
    ax.fill_between(np.arange(1,11),df_fcst_q.loc[0.25],df_fcst_median,color='#ff7f0e',alpha=.3)
    ax.plot(np.arange(1,11),df_fcst_q.loc[0.9],color='#ff7f0e',alpha=.5,linestyle='--')
    ax.plot(np.arange(1,11),df_fcst_q.loc[0.1],color='#ff7f0e',alpha=.5,linestyle='--')
    ax.legend()
    ax.set_xticks(np.arange(0,11))
    ax.margins(x=0)
    ax.grid()
    ax.set_xlabel('Lead Day')
    ax.set_ylabel('Anomaly from 90th percentile climatology [°C]')
    ax.set_ylim([-10,6])
    ax.axhline(y=0,color='k')
    ax.axvline(x=5,color='k')
    fig.suptitle('Persitent Warm Extremes Temeprature Anomalies in Forecast and Reanalysis {}-{} °N {}-{} °E'.format(lat_0,lat_1,lon_0,lon_1))
    fig.savefig(path+'/anom_files/pw_lon_{}_{}_lat_{}_{}.png'.format(lon_0,lon_1,lat_1,lat_0))
    plt.close(fig)

Plot persistent cold extremes

In [9]:
for lat_1,lat_0,lon_0,lon_1 in coordinatez:
    fig,ax = plt.subplots(figsize=(12,9))
    df_rean = pd.read_csv(path+'/anom_files/rean_lon_{}_{}_lat_{}_{}_anoms_cw.csv'.format(lon_0,lon_1,lat_1,lat_0),index_col=0)
    df_fcst = pd.read_csv(path+'/anom_files/fcst_lon_{}_{}_lat_{}_{}_anoms_cw.csv'.format(lon_0,lon_1,lat_1,lat_0),index_col=0)
    df_rean_median = df_rean.median(axis=0)
    df_fcst_median = df_fcst.median(axis=0)
    df_rean_q = df_rean.quantile([0.1,0.25,0.75,0.9])
    df_fcst_q = df_fcst.quantile([0.1,0.25,0.75,0.9])
    ax.plot(np.arange(1,11),df_rean_median,label='Reanalysis')
    ax.plot(np.arange(1,11),df_fcst_median,label='Forecast')

    ax.fill_between(np.arange(1,11),df_rean_q.loc[0.75],df_rean_median,color='#1f77b4',alpha=.3)
    ax.fill_between(np.arange(1,11),df_rean_q.loc[0.25],df_rean_median,color='#1f77b4',alpha=.3)
    ax.plot(np.arange(1,11),df_rean_q.loc[0.9],color='#1f77b4',alpha=.5,linestyle='--')
    ax.plot(np.arange(1,11),df_rean_q.loc[0.1],color='#1f77b4',alpha=.5,linestyle='--')
    
    ax.fill_between(np.arange(1,11),df_fcst_q.loc[0.75],df_fcst_median,color='#ff7f0e',alpha=.3)
    ax.fill_between(np.arange(1,11),df_fcst_q.loc[0.25],df_fcst_median,color='#ff7f0e',alpha=.3)
    ax.plot(np.arange(1,11),df_fcst_q.loc[0.9],color='#ff7f0e',alpha=.5,linestyle='--')
    ax.plot(np.arange(1,11),df_fcst_q.loc[0.1],color='#ff7f0e',alpha=.5,linestyle='--')
    ax.legend()
    ax.set_xticks(np.arange(0,11))
    ax.margins(x=0)
    ax.grid()
    ax.set_xlabel('Lead Day')
    ax.set_ylabel('Anomaly from 10th percentile climatology [°C]')
    ax.set_ylim([-6,10])
    ax.axhline(y=0,color='k')
    ax.axvline(x=5,color='k')
    fig.suptitle('Persitent Cold Extremes Temeprature Anomalies in Forecast and Reanalysis {}-{} °N {}-{} °E'.format(lat_0,lat_1,lon_0,lon_1))
    fig.savefig(path+'/anom_files/cw_lon_{}_{}_lat_{}_{}.png'.format(lon_0,lon_1,lat_1,lat_0))
    plt.close(fig)

Plot persistent average weather

In [10]:
for lat_1,lat_0,lon_0,lon_1 in coordinatez:
    fig,ax = plt.subplots(figsize=(12,9))
    df_rean = pd.read_csv(path+'/anom_files/rean_lon_{}_{}_lat_{}_{}_anoms_average.csv'.format(lon_0,lon_1,lat_1,lat_0),index_col=0)
    df_fcst = pd.read_csv(path+'/anom_files/fcst_lon_{}_{}_lat_{}_{}_anoms_average.csv'.format(lon_0,lon_1,lat_1,lat_0),index_col=0)
    df_rean_median = df_rean.median(axis=0)
    df_fcst_median = df_fcst.median(axis=0)
    df_rean_q = df_rean.quantile([0.1,0.25,0.75,0.9])
    df_fcst_q = df_fcst.quantile([0.1,0.25,0.75,0.9])
    ax.plot(np.arange(1,11),df_rean_median,label='Reanalysis')
    ax.plot(np.arange(1,11),df_fcst_median,label='Forecast')

    ax.fill_between(np.arange(1,11),df_rean_q.loc[0.75],df_rean_median,color='#1f77b4',alpha=.3)
    ax.fill_between(np.arange(1,11),df_rean_q.loc[0.25],df_rean_median,color='#1f77b4',alpha=.3)
    ax.plot(np.arange(1,11),df_rean_q.loc[0.9],color='#1f77b4',alpha=.5,linestyle='--')
    ax.plot(np.arange(1,11),df_rean_q.loc[0.1],color='#1f77b4',alpha=.5,linestyle='--')
    
    ax.fill_between(np.arange(1,11),df_fcst_q.loc[0.75],df_fcst_median,color='#ff7f0e',alpha=.3)
    ax.fill_between(np.arange(1,11),df_fcst_q.loc[0.25],df_fcst_median,color='#ff7f0e',alpha=.3)
    ax.plot(np.arange(1,11),df_fcst_q.loc[0.9],color='#ff7f0e',alpha=.5,linestyle='--')
    ax.plot(np.arange(1,11),df_fcst_q.loc[0.1],color='#ff7f0e',alpha=.5,linestyle='--')
    ax.legend()
    ax.set_xticks(np.arange(0,11))
    ax.margins(x=0)
    ax.grid()
    ax.set_xlabel('Lead Day')
    ax.set_ylabel('Anomaly from 50th percentile climatology [°C]')
    ax.set_ylim([-8,8])
    ax.axhline(y=0,color='k')
    ax.axvline(x=5,color='k')
    fig.suptitle('Peristent Average Temperature Anomalies in Forecast and Reanalysis {}-{} °N {}-{} °E'.format(lat_0,lat_1,lon_0,lon_1))
    fig.savefig(path+'/anom_files/average_lon_{}_{}_lat_{}_{}.png'.format(lon_0,lon_1,lat_1,lat_0))
    plt.close(fig)